In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

It took 5.446 hours
'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory



In [2]:
##################################################
# Load transaction 
##################################################
#input_col = []
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions')
#print ('shape of transactions',transactions.shape)
transactions = transactions.head(n = 500)

#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(transactions)
##################################################
# Convert string to datetime format
##################################################
transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))



print ('After Loading ...')
##################################################
# 善用time(時間差)來區隔交易日期時間點的行為
##################################################

def t_diff(x, time_diff):
    # x : the dataframe belong to a each specific user
    # time_diff: 2,3,4,...
    for i in range(1, time_diff):
        x['t-{}_transaction_date'.format(i)] = x['transaction_date'].shift(i)
        x['t-{}_membership_expire_date'.format(i)] = x['membership_expire_date'].shift(i)
        x['t-{}_payment_method_id'.format(i)] = x['payment_method_id'].shift(i)
        x['t-{}_payment_plan_days'.format(i)] = x['payment_plan_days'].shift(i)
        x['t-{}_plan_list_price'.format(i)] = x['plan_list_price'].shift(i)
        x['t-{}_actual_amount_paid'.format(i)] = x['actual_amount_paid'].shift(i)
        x['t-{}_is_auto_renew'.format(i)] = x['is_auto_renew'].shift(i)
        x['t-{}_is_cancel'.format(i)] = x['is_cancel'].shift(i)
    return x
def make_order_number(x):
    # for near 5
    # 1:代表最靠近現在,數字越大代表是越過去的歷史紀錄
    x['order_number_rev'] = list(reversed([i+1 for i in range(x.shape[0])]))
    return x



  0%|          | 0/9 [00:00<?, ?it/s]

reduce memory


100%|██████████| 9/9 [00:00<00:00, 20.49it/s]

After Loading ...


In [31]:
##################################################
# Main
##################################################
s = time.time()

transactions = transactions.groupby('msno').apply(t_diff, 2)
transactions = transactions.groupby('msno').apply(make_order_number)

gc.collect()
# write
path = '../input/preprocessed_data/transactions_time_diff'

output_col = ['msno','transaction_date','t-1_transaction_date','t-1_membership_expire_date',
              't-1_payment_method_id','t-1_payment_plan_days',
             't-1_plan_list_price','t-1_actual_amount_paid',
             't-1_is_auto_renew','t-1_is_cancel',
             'order_number_rev'
             ]


transactions_time_diff = transactions[output_col] 


KeyError: 'is_auto_renew'

In [33]:
output_col = ['msno','transaction_date','t-1_transaction_date','t-1_membership_expire_date',
              't-1_payment_method_id','t-1_payment_plan_days',
             't-1_plan_list_price','t-1_actual_amount_paid',
             't-1_is_auto_renew','t-1_is_cancel',
             'order_number_rev'
             ]

tran_time_diff = utils.read_multiple_csv('../../input/preprocessed_data/transactions_time_diff', output_col)
tran_time_diff = tran_time_diff.dropna() # drop columns whose oder_number == 1

tran_time_diff = tran_time_diff.head(n = 500)




  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:07<00:22,  7.63s/it]

 50%|█████     | 2/4 [00:15<00:15,  7.74s/it]

 75%|███████▌  | 3/4 [00:23<00:07,  7.86s/it]

100%|██████████| 4/4 [00:30<00:00,  7.71s/it]



In [ ]:
transactions_time_diff = transactions_time_diff.dropna() #第一筆不用無所謂, 因為第一筆訂單本來就沒有時間差的feature
transactions_time_diff[transactions_time_diff.msno == '+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=']

In [ ]:
transactions_time_diff.info()

In [34]:
def reduce_memory(df, ix_start=0):
	# There are two concepts: 1. change type, 2 drop NaN(missing) 
    #df.fillna(-1, inplace=True) # avoding value error
    if df.shape[0] <= 500:
        df_ = df.sample(1, random_state=71) # sampling for the following comoparison
    else:
        df_ = df.sample(9999, random_state=71)
    
    
    ## int
    col_int8 = []
    col_int16 = []
    col_int32 = []
    for c in tqdm(df.columns[ix_start:], miniters=20):
        print (df[c].dtype)
        if df[c].dtype=='O':
        	# '0': Objects
            continue
        elif df[c].dtype == 'datetime64[ns]':
            continue
        elif (df_[c] == df_[c].astype(np.int8)).all():
        	# Series.all():return true if all elements in Series is True
            col_int8.append(c)
        elif (df_[c] == df_[c].astype(np.int16)).all():
            col_int16.append(c)
        elif (df_[c] == df_[c].astype(np.int32)).all():
            col_int32.append(c)
    df[col_int8]  = df[col_int8].astype(np.int8)
    df[col_int16] = df[col_int16].astype(np.int16)
    df[col_int32] = df[col_int32].astype(np.int32)
    ## float
    col = [c for c in df.dtypes[df.dtypes==np.float64].index]
    df[col] = df[col].astype(np.float32)

    gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
reduce_memory(transactions_time_diff)



  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [00:00<00:00, 1809.80it/s]

reduce memory
object
datetime64[ns]
datetime64[ns]
datetime64[ns]
int8
int8
int16
int16
int8
int8
int8


In [35]:
transactions_time_diff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438 entries, 2 to 499
Data columns (total 11 columns):
msno                          438 non-null object
transaction_date              438 non-null datetime64[ns]
t-1_transaction_date          438 non-null datetime64[ns]
t-1_membership_expire_date    438 non-null datetime64[ns]
t-1_payment_method_id         438 non-null int8
t-1_payment_plan_days         438 non-null int8
t-1_plan_list_price           438 non-null int16
t-1_actual_amount_paid        438 non-null int16
t-1_is_auto_renew             438 non-null int8
t-1_is_cancel                 438 non-null int8
order_number_rev              438 non-null int8
dtypes: datetime64[ns](3), int16(2), int8(5), object(1)
memory usage: 21.0+ KB


In [36]:
transactions_time_diff[transactions_time_diff.msno == '+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=']

,msno,transaction_date,t-1_transaction_date,t-1_membership_expire_date,t-1_payment_method_id,t-1_payment_plan_days,t-1_plan_list_price,t-1_actual_amount_paid,t-1_is_auto_renew,t-1_is_cancel,order_number_rev
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-12-15,2016-11-16,2016-12-15,41,30,99,99,1,0,4
5,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2017-01-15,2016-12-15,2017-01-15,41,30,99,99,1,0,3
6,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2017-02-15,2017-01-15,2017-02-15,41,30,99,99,1,0,2
7,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2017-03-15,2017-02-15,2017-03-15,41,30,99,99,1,0,1


In [9]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 18 columns):
msno                          500 non-null object
payment_method_id             500 non-null int8
payment_plan_days             500 non-null int16
plan_list_price               500 non-null int16
actual_amount_paid            500 non-null int16
is_auto_renew                 500 non-null int8
transaction_date              500 non-null datetime64[ns]
membership_expire_date        500 non-null datetime64[ns]
is_cancel                     500 non-null int8
t-1_transaction_date          438 non-null datetime64[ns]
t-1_membership_expire_date    438 non-null datetime64[ns]
t-1_payment_method_id         438 non-null float64
t-1_payment_plan_days         438 non-null float64
t-1_plan_list_price           438 non-null float64
t-1_actual_amount_paid        438 non-null float64
t-1_is_auto_renew             438 non-null float64
t-1_is_cancel                 438 non-null float64
order_number_rev

In [13]:
transactions[transactions.msno == '+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=']

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,t-1_transaction_date,t-1_membership_expire_date,t-1_payment_method_id,t-1_payment_plan_days,t-1_plan_list_price,t-1_actual_amount_paid,t-1_is_auto_renew,t-1_is_cancel,order_number_rev
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,2016-11-16,2016-12-15,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,5
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,2016-12-15,2017-01-15,0,2016-11-16,2016-12-15,41.0,30.0,99.0,99.0,1.0,0.0,4
5,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,2017-01-15,2017-02-15,0,2016-12-15,2017-01-15,41.0,30.0,99.0,99.0,1.0,0.0,3
6,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,2017-02-15,2017-03-15,0,2017-01-15,2017-02-15,41.0,30.0,99.0,99.0,1.0,0.0,2
7,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,2017-03-15,2017-04-15,0,2017-02-15,2017-03-15,41.0,30.0,99.0,99.0,1.0,0.0,1


# days_since_the_last_transactions

In [14]:
##################################################
# Load  
##################################################
col = ['msno','plan_list_price','payment_plan_days','actual_amount_paid','payment_method_id','transaction_date','membership_expire_date']
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions', col) # 20,000,000


100%|██████████| 4/4 [00:26<00:00,  6.71s/it]


In [15]:
transactions = transactions.head(n = 500)

In [16]:
##################################################
# Convert string to datetime format
##################################################
transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))



In [17]:
def days_since_the_last_expiration(x):
    # x: each row of dataframe, series
    x['days_since_the_last_expiration'] = [i.days for i in (x.transaction_date - x['t-1_membership_expire_date'])]
    return x
def days_since_the_last_subscription(x):
    x['days_since_the_last_subscription'] = [i.days for i in (x.transaction_date - x['t-1_transaction_date'])]
    return x
def is_subscribe_early(x):
    x['is_subscribe_early'] = [1 if i <0 else 0 for i in x.days_since_the_last_expiration]
    return x
def creat_loyalty_trend(x):
    # date
    x['days_since_the_last_expiration-cumsum'] = x.days_since_the_last_expiration.cumsum()
    x['days_since_the_last_expiration_ratio'] = x.days_since_the_last_expiration.cumsum()/ [i-1 for i in list(reversed(x.order_number_rev.tolist()))]
    x['days_since_the_last_subscription_ratio'] = x.days_since_the_last_subscription.cumsum()/ [i- 1 for i in list(reversed(x.order_number_rev.tolist()))]
    x['days_since_the_last_expiration_diff'] = x.days_since_the_last_expiration - x.days_since_the_last_expiration.shift(1)
    x['days_since_the_first_subscription'] = x.days_since_the_last_subscription.cumsum()
    # payment_method
    x['do_change_payment_method'] = [1 if (p_m - t_1_p_m) != 0 else 0 for p_m, t_1_p_m in x[['payment_method_id','t-1_payment_method_id']].values]
    # plan_list_price(這次訂單,有選擇更高的價錢的方案麼)
    x['do_spend_more_money'] = [p_price - t_1_p_price for p_price, t_1_p_price in x[['plan_list_price','t-1_plan_list_price']].values]
    # payment_plan_days(這次訂單,有選擇天數更高的方案麼)
    x['do_extend_payment_days'] = [p_p_days - t_1_p_days for p_p_days, t_1_p_days in x[['payment_plan_days','t-1_payment_plan_days']].values]
    # (這次訂單,真實付的錢有比上次多麽)
    x['do_paid_more'] = [a_paid - a_paid for a_paid, a_paid in x[['actual_amount_paid','t-1_actual_amount_paid']].values]
    return x
days_since_the_last_transactions = []


In [18]:
T = 0
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))

In [40]:
df = pd.merge(train, 
transactions[(transactions.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
on='msno', 
how='left') # 此時msno就是不重複


In [41]:
c = df[df.msno == '++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=']
c

,msno,is_churn,w,plan_list_price,payment_plan_days,actual_amount_paid,payment_method_id,transaction_date,membership_expire_date
58581,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21
58582,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-07-21,2016-08-21
58583,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-08-21,2016-09-21
58584,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-09-21,2016-10-21
58585,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-10-21,2016-11-21
58586,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-11-21,2016-12-21
58587,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-12-21,2017-01-21
58588,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2017-01-21,2017-02-21
58589,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2017-02-21,2017-03-21


In [42]:
#tran_time_diff = transactions_time_diff
d = tran_time_diff[tran_time_diff.msno == '++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=']
d

,msno,transaction_date,t-1_transaction_date,t-1_membership_expire_date,t-1_payment_method_id,t-1_payment_plan_days,t-1_plan_list_price,t-1_actual_amount_paid,t-1_is_auto_renew,t-1_is_cancel,order_number_rev
243,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-07-21,2016-06-22,2016-07-21,41,30,99,99,1,0,9
244,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-08-21,2016-07-21,2016-08-21,41,30,99,99,1,0,8
245,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-09-21,2016-08-21,2016-09-21,41,30,99,99,1,0,7
246,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-10-21,2016-09-21,2016-10-21,41,30,99,99,1,0,6
247,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-11-21,2016-10-21,2016-11-21,41,30,99,99,1,0,5
248,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-12-21,2016-11-21,2016-12-21,41,30,99,99,1,0,4
249,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-01-21,2016-12-21,2017-01-21,41,30,99,99,1,0,3
250,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-02-21,2017-01-21,2017-02-21,41,30,99,99,1,0,2
251,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-03-21,2017-02-21,2017-03-21,41,30,99,99,1,0,1


In [43]:
qqqq = pd.merge(c,d , on =['msno','transaction_date'])
qqqq

,msno,is_churn,w,plan_list_price,payment_plan_days,actual_amount_paid,payment_method_id,transaction_date,membership_expire_date,t-1_transaction_date,t-1_membership_expire_date,t-1_payment_method_id,t-1_payment_plan_days,t-1_plan_list_price,t-1_actual_amount_paid,t-1_is_auto_renew,t-1_is_cancel,order_number_rev
0,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-07-21,2016-08-21,2016-06-22,2016-07-21,41,30,99,99,1,0,9
1,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-08-21,2016-09-21,2016-07-21,2016-08-21,41,30,99,99,1,0,8
2,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-09-21,2016-10-21,2016-08-21,2016-09-21,41,30,99,99,1,0,7
3,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-10-21,2016-11-21,2016-09-21,2016-10-21,41,30,99,99,1,0,6
4,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-11-21,2016-12-21,2016-10-21,2016-11-21,41,30,99,99,1,0,5
5,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-12-21,2017-01-21,2016-11-21,2016-12-21,41,30,99,99,1,0,4
6,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2017-01-21,2017-02-21,2016-12-21,2017-01-21,41,30,99,99,1,0,3
7,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2017-02-21,2017-03-21,2017-01-21,2017-02-21,41,30,99,99,1,0,2


In [44]:
qqqq = qqqq.groupby('msno').apply(days_since_the_last_expiration)
qqqq = qqqq.groupby('msno').apply(days_since_the_last_subscription)
qqqq = qqqq.groupby('msno').apply(is_subscribe_early)
qqqq = qqqq.groupby('msno').apply(creat_loyalty_trend)
output_col = ['msno','w',
              'days_since_the_last_expiration',
            'days_since_the_last_subscription','is_subscribe_early','order_number_rev',
            'days_since_the_last_expiration-cumsum',
            'days_since_the_last_expiration_ratio',
            'days_since_the_last_subscription_ratio',
            'days_since_the_last_expiration_diff',
            'days_since_the_first_subscription',
                  'do_change_payment_method',
                  'do_spend_more_money',
                  'do_extend_payment_days',
                  'do_paid_more',
            ]
qqqq = qqqq[output_col]
qqqq.reset_index(drop = True)

,msno,w,days_since_the_last_expiration,days_since_the_last_subscription,is_subscribe_early,order_number_rev,days_since_the_last_expiration-cumsum,days_since_the_last_expiration_ratio,days_since_the_last_subscription_ratio,days_since_the_last_expiration_diff,days_since_the_first_subscription,do_change_payment_method,do_spend_more_money,do_extend_payment_days,do_paid_more
0,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,29,0,9,0,0.0,29.000000,NaN,29,0,0.0,0.0,0.0
1,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,31,0,8,0,0.0,30.000000,0.0,60,0,0.0,0.0,0.0
2,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,31,0,7,0,0.0,30.333333,0.0,91,0,0.0,0.0,0.0
3,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,30,0,6,0,0.0,30.250000,0.0,121,0,0.0,0.0,0.0
4,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,31,0,5,0,0.0,30.400000,0.0,152,0,0.0,0.0,0.0
5,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,30,0,4,0,0.0,30.333333,0.0,182,0,0.0,0.0,0.0
6,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,31,0,3,0,0.0,30.428571,0.0,213,0,0.0,0.0,0.0
7,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,31,0,2,0,0.0,30.500000,0.0,244,0,0.0,0.0,0.0


In [ ]:
qqqq.shape

In [25]:
reduce_memory(qqqq)

100%|██████████| 15/15 [00:00<00:00, 1550.38it/s]

object
int64
int64
int64
int64
int8
int64
float64
float64
float64
int64
int64
float64
float64
float64


ValueError: Cannot convert non-finite values (NA or inf) to integer